In [146]:
import re
import math
import warnings
import tqdm
import json
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import optimize


warnings.filterwarnings("ignore")

In [147]:
data1 = pd.read_excel('2a数据_已替换时间戳_已矫正.xlsx')
with open("best_labels.json", "r") as json_file:
    group_targets = json.load(json_file)
new_column = pd.Series(group_targets["best_labels"], name="组号")
data1 = pd.concat([data1, new_column], axis=1)

In [148]:
data1

,ID,发病到首次影像检查时间间隔,入院首次影像检查流水号,ED_volume.0,随访1流水号,ED_volume.1,随访2流水号,ED_volume.2,随访3流水号,ED_volume.3,...,ED_volume.4,随访5流水号,ED_volume.5,随访6流水号,ED_volume.6,随访7流水号,ED_volume.7,随访8流水号,ED_volume.8,组号
0,sub001,2.5,9000,48919,29766.0,57898.0,475590.0,81747.0,935055.0,107793.0,...,126558.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
1,sub002,3.0,10800,23526,53724.0,23390.0,249199.0,28415.0,1612859.0,34868.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
2,sub003,2.0,7200,32621,34281.0,41292.0,142545.0,49767.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
3,sub004,1.0,3600,22191,61107.0,18956.0,301843.0,9540.0,819004.0,5204.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
4,sub005,5.0,18000,47392,95283.0,23182.0,352595.0,12898.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,sub156,5.5,19800,8968,89575.0,12860.0,452217.0,13716.0,1055942.0,12709.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,sub157,0.5,1800,409,4920.0,3742.0,25620.0,53311.0,72240.0,29545.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,sub158,2.8,10080,17948,66180.0,21099.0,256680.0,44069.0,1111140.0,64243.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158,sub159,3.0,10800,14430,95520.0,19019.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
# 按组号分组
grouped_data = data1.groupby("组号")
grouped_dataframes = {}
for group, group_df in grouped_data:
    grouped_dataframes[group] = group_df.drop(["发病到首次影像检查时间间隔","组号"], axis = 1).reset_index().drop(["index"], axis = 1)

In [142]:
grouped_dataframes[group]

,ID,入院首次影像检查流水号,ED_volume.0,随访1流水号,ED_volume.1,随访2流水号,ED_volume.2,随访3流水号,ED_volume.3,随访4流水号,ED_volume.4,随访5流水号,ED_volume.5,随访6流水号,ED_volume.6,随访7流水号,ED_volume.7,随访8流水号,ED_volume.8
0,sub002,10800,23526,53724.0,23390.0,249199.0,28415.0,1612859.0,34868.0,4029091.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sub004,3600,22191,61107.0,18956.0,301843.0,9540.0,819004.0,5204.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sub005,18000,47392,95283.0,23182.0,352595.0,12898.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sub009,2412,18799,144202.0,22191.0,238178.0,48539.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sub012,7200,15254,41554.0,16723.0,104632.0,13818.0,257805.0,20788.0,622277.0,22086.0,1732597.0,55714.0,4578517.0,1099.0,NaN,NaN,NaN,NaN
5,sub014,3600,12291,84216.0,19434.0,861131.0,31092.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,sub018,1800,23890,56056.0,38354.0,138317.0,45387.0,849773.0,64084.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,sub019,32400,11107,65955.0,16594.0,153490.0,18649.0,1068844.0,27745.0,2901171.0,1549.0,6423754.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
8,sub020,2412,160,21956.0,527.0,130705.0,233.0,2623363.0,0.0,6005977.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [143]:
# 取XY值
grouped_dataframes_XY = {}
for g in grouped_dataframes:
    x=[]
    y=[]
    ID=[]
    datapp=pd.DataFrame()
    for i in range(len(list(grouped_dataframes[g].columns))):
        if list(grouped_dataframes[g].columns)[i][-3:]=='流水号':
            for j in range(len(grouped_dataframes[g][list(grouped_dataframes[g].columns)[i]])):
                if j <=99 and not math.isnan(grouped_dataframes[g][list(grouped_dataframes[g].columns)[i]][j]) and not math.isnan(grouped_dataframes[g][list(grouped_dataframes[g].columns)[i+1]][j]):
                    x.append(grouped_dataframes[g][list(grouped_dataframes[g].columns)[i]][j]/3600)
                    y.append(grouped_dataframes[g][list(grouped_dataframes[g].columns)[i+1]][j]/1000)
                    ID.append(grouped_dataframes[g]['ID'][j])
    datapp['ID']=ID.copy()
    datapp['X']=x.copy()
    datapp['Y']=y.copy()
    grouped_dataframes_XY[g] = {"datapp": datapp.copy()}

In [145]:
grouped_dataframes_XY

{0.0: {'datapp':         ID            X       Y
  0   sub008     2.000000  14.353
  1   sub010     1.000000  15.272
  2   sub013     3.000000   9.694
  3   sub017     2.000000  11.533
  4   sub023     2.000000  42.734
  5   sub024     5.000000   9.317
  6   sub059     1.000000   1.396
  7   sub078     5.000000   2.679
  8   sub079     1.000000   7.317
  9   sub088    10.000000  23.528
  10  sub093     0.500000   4.923
  11  sub008    10.245556  32.889
  12  sub010    15.000000  22.355
  13  sub013    26.464167   9.249
  14  sub017    14.870278  22.075
  15  sub023    28.161667  44.699
  16  sub024    29.363889   7.296
  17  sub059    46.111389   3.226
  18  sub078    31.239722   0.552
  19  sub079    27.847222   6.525
  20  sub088    34.757500  13.051
  21  sub093     3.577778   3.943
  22  sub008    98.245833  51.887
  23  sub010    39.332500  23.424
  24  sub013   194.313333  15.231
  25  sub017   181.805833  44.436
  26  sub023   221.032222  41.808
  27  sub024   297.228056   1.124

In [144]:
#线性
def liner_func(x,a,b):
    return a*x+b

#二次
def erchi_func(x,a,b,c):
    return a*x**2+b*x+c

#三次
def sanchi_func(x,a,b,c,d):
    return a*x**3+b*x**2+c*x+d

# 三角函数
def trig_func(x,a,b,c):
    return a*np.sin(x)+b*np.cos(x)+c
# 指数曲线
def target_func(x, a, b, c):
    return a * np.exp(-x / b) + c
# 对数函数
def hyp_func(x, a,b):
    return a*np.log(x)+b

# 定义高斯函数
def gaussian(x, a, b, c):
    return a * np.exp(-(x - b) ** 2 / (2 * c ** 2))

#定义Logistic函数
def logistic(x,a,b,c):
    return  a / (1 + np.exp(-b * (x - c)))


def __sst(y_no_fitting):
    """
    计算SST(total sum of squares) 总平方和
    :param y_no_predicted: List[int] or array[int] 待拟合的y
    :return: 总平方和SST
    """
    y_mean = sum(y_no_fitting) / len(y_no_fitting)
    s_list =[(y - y_mean)**2 for y in y_no_fitting]
    sst = sum(s_list)
    return sst


def __ssr(y_fitting, y_no_fitting):
    """
    计算SSR(regression sum of squares) 回归平方和
    :param y_fitting: List[int] or array[int]  拟合好的y值
    :param y_no_fitting: List[int] or array[int] 待拟合y值
    :return: 回归平方和SSR
    """
    y_mean = sum(y_no_fitting) / len(y_no_fitting)
    s_list =[(y - y_mean)**2 for y in y_fitting]
    ssr = sum(s_list)
    return ssr


def __sse(y_fitting, y_no_fitting):
    """
    计算SSE(error sum of squares) 残差平方和
    :param y_fitting: List[int] or array[int] 拟合好的y值
    :param y_no_fitting: List[int] or array[int] 待拟合y值
    :return: 残差平方和SSE
    """
    s_list = [(y_fitting[i] - y_no_fitting[i])**2 for i in range(len(y_fitting))]
    sse = sum(s_list)
    return sse


def goodness_of_fit(y_fitting, y_no_fitting):
    """
    计算拟合优度R^2
    :param y_fitting: List[int] or array[int] 拟合好的y值
    :param y_no_fitting: List[int] or array[int] 待拟合y值
    :return: 拟合优度R^2
    """
    SSR = __ssr(y_fitting, y_no_fitting)
    SST = __sst(y_no_fitting)
    rr = SSR /SST
    return rr
def selected_func(you,model,model_select):
    maxyou=max(you)
    for s in range(len(you)):
        if you[s]==maxyou:
            
            return model[s],model_select[s]
        
    

def is_non_real(x):
    # 判断复数是否为非实数
    return math.isinf(x) or math.isnan(x)

def sgn(x):
    if x < 0:
        return -1
    elif x == 0:
        return 0
    else:
        return 1
        

# 求解方程 f(x) = y
def solve_equation(f,y):
    equation = lambda x: f(x) - y
    x_initial_guess = 0  # 初始猜测值
    x_solution = fsolve(equation, x_initial_guess)
    return x_solution

for g in grouped_dataframes_XY:
    data=grouped_dataframes_XY[g]['datapp']
    x=data['X']
    y=data['Y']

    Set={}
    youSet={}
    resSet={}
    resTime={}
    you=[]
    model=[]
    model_select=[]
    you_index={}
    RMR=[]
    try:
        a1 = np.polyfit(x, y, 1)#线性
        you1 = goodness_of_fit([liner_func(x[p],a1[0],a1[1]) for p in range(len(x))],y)
        you.append(you1)
        rmr1=[round(liner_func(x[p],a1[0],a1[1])-y[p],4) for p in range(len(x))]
        model.append({'回归类型':'线性回归','回归系数':a1,'残差':rmr1,'ID':ID})
        you_index['线性回归']=you1
        model_select.append((liner_func,a1))
    except:
        pass

    try:
        a2 = np.polyfit(x, y, 2)#二次
        you2 = goodness_of_fit([erchi_func(x[p],a2[0],a2[1],a2[2]) for p in range(len(x))],y)
        rmr2=[round(erchi_func(x[p],a2[0],a2[1],a2[2])-y[p],4) for p in range(len(x))]
        you.append(you2)
        model.append({'回归类型':'二次函数回归','回归系数':a2,'残差':rmr2,'ID':ID})
        you_index['二次函数回归']=you2
        model_select.append((erchi_func,a2))
    except:
        pass

    try:
        a3 = np.polyfit(x, y, 3)#三次
        you3 = goodness_of_fit([sanchi_func(x[p],a3[0],a3[1],a3[2],a3[3]) for p in range(len(x))],y)
        rmr3=[round(sanchi_func(x[p],a3[0],a3[1],a3[2],a3[3])-y[p],4) for p in range(len(x))]
        you.append(you3)
        model.append({'回归类型':'三次函数回归','回归系数':a3,'残差':rmr3,'ID':ID})
        you_index['三次函数回归']=you3
        model_select.append((sanchi_func,a3))
    except:
        pass

    #拟合三角函数模型
    try:
        a4,_=optimize.curve_fit(trig_func,x,y)
        you4 = goodness_of_fit([trig_func(x[p],a4[0],a4[1],a4[2]) for p in range(len(x))],y)
        rmr4=[round(trig_func(x[p],a4[0],a4[1],a4[2])-y[p],4) for p in range(len(x))]
        you.append(you4)
        model.append({'回归类型':'三角函数回归','回归系数':a4,'残差':rmr4,'ID':ID})
        you_index['三角函数回归']=you4
        model_select.append((trig_func,a4))

    except:
        pass

    #拟合指数函数模型
    try:
        a5,_=optimize.curve_fit(target_func,x,y)
        you5 = goodness_of_fit([target_func(x[p], a5[0], a5[1], a5[2]) for p in range(len(x))],y)
        rmr5=[round(target_func(x[p], a5[0], a5[1], a5[2])-y[p],4) for p in range(len(x))]
        you.append(you5)
        model.append({'回归类型':'指数函数回归','回归系数':a5,'残差':rmr5,'ID':ID})
        you_index['指数函数回归']=you5
        model_select.append((target_func,a5))
    except:
        pass

    #拟合对数函数模型
    try:
        a6,_=optimize.curve_fit(hyp_func,x,y)
        you6 = goodness_of_fit([hyp_func(x[p], a6[0],a6[1]) for p in range(len(x))],y)
        rmr6=[round(hyp_func(x[p], a6[0],a6[1])-y[p],4) for p in range(len(x))]
        you.append(you6)
        model.append({'回归类型':'对数函数回归','回归系数':a6,'残差':rmr6,'ID':ID})
        you_index['对数函数回归']=you6
        model_select.append((hyp_func,a6))
    except:
        pass
    try:
        popt, pcov = optimize.curve_fit(gaussian, x, y)
        # 提取拟合参数
        a_fit, b_fit, c_fit = popt

        you7 = goodness_of_fit([gaussian(x[p], a_fit,b_fit,c_fit) for p in range(len(x))],y)
        rmr7=[round(gaussian(x[p], a_fit,b_fit,c_fit)-y[p],4) for p in range(len(x))]
        you.append(you7)
        model.append({'回归类型':'高斯函数回归','回归系数':popt,'残差':rmr7,'ID':ID})
        you_index['高斯函数回归']=you7
        model_select.append((gaussian,popt))
    except:
        pass

    try:
        popt, pcov = optimize.curve_fit(logistic, x, y)
        # 提取拟合参数
        a_fit, b_fit, c_fit = popt
        you8 = goodness_of_fit([logistic(x[p], a_fit,b_fit,c_fit) for p in range(len(x))],y)

        rmr8=[round(logistic(x[p], a_fit,b_fit,c_fit)-y[p],4) for p in range(len(x))]
        you.append(you8)
        model.append({'回归类型':'logistic函数回归','回归系数':popt,'残差':rmr8,'ID':ID})
        you_index['logistic函数回归']=you8
        model_select.append((logistic,popt))

    except:
        pass

    # model=[f'线性回归，系数为{a1}',f'二次函数回归，系数为{a2}',f'三次函数回归，系数为{a3}',f'三角函数回归，系数为{a4}',f'指数函数回归，系数为{a5}',f'对数函数回归，系数为{a6}']
    res,model_=selected_func(you,model,model_select)

    # Set[index]=res#选择函数模型，例如：1用三角函数
    # youSet[index]=you_index#拟合优度

    print(you_index)
    p=0 #取最大
    for i in range(len(list(you_index.keys()))):
        if you_index[list(you_index.keys())[i]]>you_index[list(you_index.keys())[p]]:
            p=i

    print(model[p])
    ID=model[p]['ID']
    RMR=model[p]['残差']
    SET_RMR={}
    for i in range(len(ID)):
        SET_RMR[ID[i]]=[]
    for i in range(len(ID)):
        SET_RMR[ID[i]].append(model[p]['残差'][i])
    datasave=pd.DataFrame(columns=['ID','残差'])
    datasave['ID']=SET_RMR.keys()
    datasave['残差']=SET_RMR.values()
    datasave.to_excel(f'Q22填表答案_向量结果_{g}.xlsx',index=False)
    datasave['残差']=datasave['残差'].apply(lambda x: sum(x) / len(x))
    datasave.to_excel(f'Q22填表答案_单值结果_{g}.xlsx',index=False)

{'线性回归': 0.057132792262263456, '二次函数回归': 0.07462673914341282, '三次函数回归': 0.10627076216752833, '三角函数回归': 0.03671540632047492, '指数函数回归': 0.019713591438636775, '对数函数回归': 0.00023245405690029483, '高斯函数回归': 0.5173604309340998, 'logistic函数回归': 1.1545708254504762e-20}
{'回归类型': '高斯函数回归', '回归系数': array([18.97817648, 18.19277866, 18.32229878]), '残差': [-1.5105, -3.0525, 3.7631, 1.3095, -29.8915, 5.3274, 10.8235, 11.9654, 4.9025, -6.3553, 6.9832, -15.6147, -3.6628, 7.8906, -3.4063, -28.3319, 8.4632, 2.718, 14.1762, 9.9932, -0.4394, 9.8637, -51.8856, -13.6698, -15.231, -44.436, -41.808, -1.124, -1.488, -2.378, -9.754, -29.34, 10.5608, -34.914, -10.274, -7.072, -34.853, -40.009, 0.0, -1.794, -4.8811, 0.0, -1.754, -30.197, -0.053, -3.258, -3.868, -0.377], 'ID': ['sub002', 'sub004', 'sub005', 'sub009', 'sub012', 'sub014', 'sub018', 'sub019', 'sub020', 'sub002', 'sub004', 'sub005', 'sub009', 'sub012', 'sub014', 'sub018', 'sub019', 'sub020', 'sub002', 'sub004', 'sub005', 'sub009', 'sub012', 'sub014', 'sub

IndexError: list index out of range